# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: John Murrowood
#### Student ID: S3923075

Date: 20/9/2022

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy
* sys
* os
* nltk
* future
* itertools

## Introduction
The following blocks of code will import a set of job advertisments. Only the webindex or these advertisments and their descriptions will be taken for processing from these documents. The aim is to then process the descriptions into list of tokens for each document to be easily processesd into a classification model to predict the job category that these jobs belong to. These jobs are first processed to ensure any html scraping inconsistencies are removed and then tokenised to only capture words that follow the particular format of the given regex expression. This will delete any symbols or expressions that could be errors or are not words. Other words are then deleted along the preprocessing process that are unnessisary for the model so it can be more accurate. This includes removing words with only one letter, removing stopwords that are irrelevant to job context, making all letetrs lowercase to avoid repeated words, and removing words that freuquently occurr and occur only once. 

The tokenised documents will then be saved into an output directory so they can be used later in classification models and a txt file of the vocabulary will also be saved so vector representations of the models can also be easily made in the future.

## Importing libraries 

In [127]:
# Imported libraries used for task 1

import pandas as pd
import numpy as np
import os
import nltk
import re
import sys
from nltk.tokenize import RegexpTokenizer
from __future__ import division
from itertools import chain
from nltk.probability import *



### 1.1 Examining and loading data
- Examine the data folder, including the categories and job advertisment txt documents, etc. Explain your findings here, e.g., number of folders and format of txt files, etc.
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.


In [128]:
# The following code block reades in all the job txt files and extracts the webindex number of each job as well as the description.
# The description of each job is saved as a list of strings in description_txt while the webindex is saved in description_ids.

dir_path = "./data" # The directory path to access joadvertisments in data folder of given zip folder.
description_ids = [] # list to store the advertisment webindex strings
description_txts = [] # list to store the raw text of advertisment job descriptions
webIndex_pattern = r"Webindex: (\d+)" # Regex pattern used to extract webindex pattern
for file in sorted(os.listdir(dir_path)): # Loop through each of the job category files to access all job advertisments
    if file == ".DS_Store": #Do not look in DS_Store file on mac
        pass
    else:
        for filename in sorted(os.listdir((dir_path+"/"+file))): # load job advertisments in ascending order of txt file names
            if filename.endswith(".txt"): # Ensure only txt files are opened
                path = os.path.join(dir_path+"/"+file+"/"+filename) # This gives the path to the exact filename of each txt documents to open
                with open(path,"r",encoding= 'unicode_escape') as f: # open the txt file
                    text_whole = f.read() # Read in all the text in the advertisment
                    description_ids.append(re.search(webIndex_pattern, text_whole).groups()[0]) # Extract the web index from the advertisent
                                                                                                # and append to ids list
                    text_description =  text_whole.split("\nDescription: ")[1] # take only the job description text of the job advertisment
                    description_txts.append(text_description) # Append description to list of description text for further processing
                    f.close()


In [129]:
# Remove any incorrect words with br or brbr infront of some of the words
pattern = r"(?:br{1,2})[A-Z]\w*" # Pattern for extracting unwanted br or brbr prefixes to some words
for a in range(len(description_txts)): # Loop through advertisment descriptions
    i = description_txts[a] # load in one description at a time for checking
    if re.findall(pattern, i) != []: # If brbr not found no need to do any fixing
        for j in re.findall(pattern, i): # Loop through each found instance of br or brbr
            i = i.replace(j, j[2:]) # If br prefix then remove
            if 'br' in i: # If second br still in word
                i = i.replace(j, j[2:]) # Remove second instance of br prefix
        description_txts[a] = i # Add fixed text body back and replace old body
        print(i) # Check to ensure fix has worked

Position: Staff Nurse  RGN will also consider Newly Qualified Location: Selby Salary: **** per hour plus overtime rate Job Description: I am currently looking to recruit a qualified RGN to work for a service based within a rural location. The service is CQC compliant and part of a Yorkshire based Healthcare Company Job Requirements:Responsible for the assessment of care/support needs of service usersDevelopment and implementation of care programmes Working alongside other nurses reporting to the Manager Skills/ Qualifications:Registered Nurse  RGN will also consider newly qualified Desire to make a difference to people Passionate at delivering services that enhance lives Benefits:Salary **** per hour plus overtime rate Holiday entitlement EXCELLENT career progression and training opportunities Picturesque working environment For more information on how to apply for this fantastic opportunity please contact Shona Blackburn on or email a copy of your up to date CV for immediate attention

### 1.2 Pre-processing data
Perform the required text pre-processing steps.

In [131]:
# Tokenisation using given regular expression:
pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?" # Given regex expression to perform tokenisation
description_txt_token = [] # Establish a list of lists of tokenised words in each description
tokenizer = RegexpTokenizer(pattern) # Initialise tokeniser using regex pattern
for desc in description_txts: # Loop through all desciptions
    tokens = tokenizer.tokenize(desc) # Get tokens from each job description
    description_txt_token.append(tokens) # Append list of tokens for each document to tokenised documents list

# Print out random document to check tokenisation worked
print("Description of", description_ids[488]) 
print(description_txt_token[488])
print("Number of tokens found in job description: ", len(description_txt_token[1]))



Description of 68559978
['Position', 'Staff', 'Nurse', 'RGN', 'will', 'also', 'consider', 'Newly', 'Qualified', 'Location', 'Selby', 'Salary', 'per', 'hour', 'plus', 'overtime', 'rate', 'Job', 'Description', 'I', 'am', 'currently', 'looking', 'to', 'recruit', 'a', 'qualified', 'RGN', 'to', 'work', 'for', 'a', 'service', 'based', 'within', 'a', 'rural', 'location', 'The', 'service', 'is', 'CQC', 'compliant', 'and', 'part', 'of', 'a', 'Yorkshire', 'based', 'Healthcare', 'Company', 'Job', 'Requirements', 'Responsible', 'for', 'the', 'assessment', 'of', 'care', 'support', 'needs', 'of', 'service', 'usersDevelopment', 'and', 'implementation', 'of', 'care', 'programmes', 'Working', 'alongside', 'other', 'nurses', 'reporting', 'to', 'the', 'Manager', 'Skills', 'Qualifications', 'Registered', 'Nurse', 'RGN', 'will', 'also', 'consider', 'newly', 'qualified', 'Desire', 'to', 'make', 'a', 'difference', 'to', 'people', 'Passionate', 'at', 'delivering', 'services', 'that', 'enhance', 'lives', 'Bene

In [132]:
# convert all tokens to lower case:
description_txt_token_lower = [] # Initialise a list for all lower case tokens
for desc in description_txt_token: # loop through each tokenised desription
    for i in range(len(desc)): # loop through each token
        desc[i] = desc[i].lower() # Force every word to be all lower case
    description_txt_token_lower.append(desc) # Append list of lowercase tokens to lowered tokens list

# Print out the lowered tokens to ensure it has worked
print("Description of", description_ids[488])
print(description_txt_token_lower[488])
print("Number of tokens found in job description: ", len(description_txt_token_lower[1]))

Description of 70757636
['you', 'will', 'be', 'responsible', 'for', 'the', 'efficient', 'running', 'of', 'the', 'accounting', 'function', 'of', 'the', 'business', 'by', 'implementing', 'procedures', 'and', 'controls', 'to', 'ensure', 'the', 'accuracy', 'of', 'the', 'management', 'information', 'presented', 'to', 'the', 'directors', 'to', 'establish', 'and', 'maintain', 'financial', 'policies', 'and', 'management', 'information', 'systems', 'and', 'to', 'liaise', 'with', 'management', 'colleagues', 'on', 'all', 'aspects', 'of', 'finance', 'to', 'provide', 'a', 'high', 'quality', 'support', 'service', 'to', 'the', 'directors', 'and', 'officers', 'of', 'the', 'company', 'duties', 'to', 'include', 'leading', 'a', 'team', 'of', 'staff', 'responsible', 'for', 'the', 'production', 'of', 'management', 'accounts', 'creating', 'implementing', 'and', 'monitoring', 'processes', 'and', 'procedures', 'for', 'the', 'creation', 'of', 'monthly', 'forecasts', 'preparing', 'detailed', 'weekly', 'monthly'

In [133]:
# Remove words with length less than 2:
min_word_length = 2 # Initialise the minimum word length 
description_txt_token_lower_rem = [] # Initialise list to add new tokens with words removed
for desc in description_txt_token_lower: # Loop through each advertisment tokenised list
    for word in desc: # Loop through each token
        if len(word) < min_word_length: # Check if word is less than the minimum length
            desc.remove(word)
    description_txt_token_lower_rem.append(desc)

# Check removal worked
print("Description of", description_ids[1])
print(description_txt_token_lower_rem[1])
print("Number of tokens found in job description with initial removal: ", len(description_txt_token_lower_rem[1]))
        

Description of 70757636
['you', 'will', 'be', 'responsible', 'for', 'the', 'efficient', 'running', 'of', 'the', 'accounting', 'function', 'of', 'the', 'business', 'by', 'implementing', 'procedures', 'and', 'controls', 'to', 'ensure', 'the', 'accuracy', 'of', 'the', 'management', 'information', 'presented', 'to', 'the', 'directors', 'to', 'establish', 'and', 'maintain', 'financial', 'policies', 'and', 'management', 'information', 'systems', 'and', 'to', 'liaise', 'with', 'management', 'colleagues', 'on', 'all', 'aspects', 'of', 'finance', 'to', 'provide', 'high', 'quality', 'support', 'service', 'to', 'the', 'directors', 'and', 'officers', 'of', 'the', 'company', 'duties', 'to', 'include', 'leading', 'team', 'of', 'staff', 'responsible', 'for', 'the', 'production', 'of', 'management', 'accounts', 'creating', 'implementing', 'and', 'monitoring', 'processes', 'and', 'procedures', 'for', 'the', 'creation', 'of', 'monthly', 'forecasts', 'preparing', 'detailed', 'weekly', 'monthly', 'and', '

In [134]:
# Importing stopwords file
stop_file = "stopwords_en.txt"
with open(stop_file, "r", encoding= 'unicode_escape') as f: # open the txt file
    stop_words = f.read()


In [135]:
# Removing stopwords
description_txt_token_no_stop = [] # Initialise tokens list with stopwords removed
for desc in description_txt_token_lower_rem: # Loop through existing tokens
    description_txt_token_no_stop.append([t for t in desc if t not in stop_words.split("\n")]) # Add tokens to new list if they are not a stop word
# Check if stopword removal worked
print("Description of", description_ids[1])
print(description_txt_token_no_stop[1])
print("Number of tokens found in job description with initial removal: ", len(description_txt_token_no_stop[1]))

Description of 70757636
['responsible', 'efficient', 'running', 'accounting', 'function', 'business', 'implementing', 'procedures', 'controls', 'ensure', 'accuracy', 'management', 'information', 'presented', 'directors', 'establish', 'maintain', 'financial', 'policies', 'management', 'information', 'systems', 'liaise', 'management', 'colleagues', 'aspects', 'finance', 'provide', 'high', 'quality', 'support', 'service', 'directors', 'officers', 'company', 'duties', 'include', 'leading', 'team', 'staff', 'responsible', 'production', 'management', 'accounts', 'creating', 'implementing', 'monitoring', 'processes', 'procedures', 'creation', 'monthly', 'forecasts', 'preparing', 'detailed', 'weekly', 'monthly', 'quarterly', 'cash', 'flow', 'forecasts', 'preparing', 'detailed', 'annual', 'quarterly', 'monthly', 'profit', 'forecasts', 'preparing', 'detailed', 'monthly', 'financial', 'statements', 'include', 'profit', 'loss', 'accounts', 'balance', 'sheet', 'variance', 'analysis', 'commentaries'

In [136]:
# Create a term and doc frequency to remove low frequency and high frequncy words
words = list(chain.from_iterable(description_txt_token_no_stop))
words_2 = list(chain.from_iterable([set(description) for description in description_txt_token_no_stop]))
doc_fd = FreqDist(words_2) # compute term frequency for each unique word/type
term_fd = FreqDist(words)


In [137]:
# Remove words that appear only once
lessFreqWords = set(term_fd.hapaxes())
description_txt_token_freq = [] # Initilise list for filtered tokens
for desc in description_txt_token_no_stop:
    description_txt_token_freq.append([t for t in desc if t not in lessFreqWords]) # Add words to list that appear more than once only


In [138]:
# Remove top 50 most occuring words
moreFreqwords = doc_fd.most_common(50)
most_words = []
for i in moreFreqwords:
    most_words.append(i[0])

for i in range(len(description_txt_token_no_stop)):
    description_txt_token_freq[i] = [t for t in description_txt_token_freq[i] if t not in most_words] # Delete words that are in top 50 most occuring

In [139]:
# Check that word filter worked
print(description_txt_token_freq[478])

['description', 'inspired', 'selections', 'optical', 'assistant', 'oxfordshire', 'locally', 'level', 'customer', 'established', 'years', 'ago', 'friendly', 'professional', 'highly', 'competitive', 'prices', 'practices', 'fully', 'qualified', 'dispensing', 'opticians', 'hand', 'members', 'association', 'british', 'dispensing', 'opticians', 'small', 'independent', 'group', 'prices', 'larger', 'chains', 'patients', 'enjoy', 'worlds', 'committed', 'care', 'staff', 'respected', 'loyal', 'patients', 'inspired', 'selections']


## Saving required outputs
Save the vocabulary, bigrams and job advertisment txt as per spectification.
- vocab.txt

In [140]:
# Saving job advertisement outputs to text files for each ad
os.mkdir('output/') # Make a new output directory to store tokenised txt files
for i in range(len(description_ids)): # loop through each tokenised list
    with open('output/'+description_ids[i], 'w') as output:
        output.write(' '.join([str(elem) for elem in description_txt_token_freq[i]])) #Write each tokenised word seperated by a space


In [141]:
# Saving vocab txt file with all stopwords removed as specified
file_name = 'vocab.txt'
words = sorted(list(chain.from_iterable(description_txt_token_freq))) #Make each word appear in alphabetical order
vocab = list(dict.fromkeys(words)) # Make vocab list 
with open(file_name, 'w') as f:
    for i in range(len(vocab)):
        f.write(f"{vocab[i]}:{i}\n") # Save each word with index on txt file


## Summary
The vocab file was able to be outputted in the correct formnat of word:index and the tokenised descriptions were each saved to their own txt file in a directory called output. Each txt file was given the name of its webindex so the webindex can easily be matched to the description for future reference. It could be seen that removing stop words removed a large portion of the tokenised words showing this is a very important step because these words have very little context on the point of the description and therefore arent helpful in the classification model. The regex pattern was also very effective at ensuring only words were captured with no numbers or other symbols prersent in some of the documents.